In [1]:
%pip install anthropic --upgrade

Note: you may need to restart the kernel to use updated packages.


## Step 1: Set up the Environment
## Description: Install the required libraries and set up the Anthropic API client.

In [3]:
import anthropic

client = anthropic.Client()
MODEL_NAME = "claude-3-opus-20240229"

## Step 2: Define the Client-side Tools

### Description
Define the tools that the chatbot will use to assist customers, such as checking availability and booking vacations.

### Input Parameters
- **destination:** The destination for the vacation package (e.g., "Maldives").
- **dates:** The dates for the vacation package in YYYY-MM-DD format.
- **customer_name:** The name of the customer making the booking.
- **email:** The email address of the customer.
- **phone:** The phone number of the customer.

In [12]:
# Step 2: Define the client-side tools
tools = [
    {
        "name": "check_availability",
        "description": "Checks the availability of vacation packages for the given destination and dates.",
        "input_schema": {
            "type": "object",
            "properties": {
                "destination": {
                    "type": "string",
                    "description": "The destination for the vacation package (e.g., 'Maldives')."
                },
                "dates": {
                    "type": "string",
                    "description": "The dates for the vacation package in YYYY-MM-DD format."
                }
            },
            "required": ["destination", "dates"]
        }
    },
    {
        "name": "book_vacation",
        "description": "Books a vacation package for the given destination and dates.",
        "input_schema": {
            "type": "object",
            "properties": {
                "destination": {
                    "type": "string",
                    "description": "The destination for the vacation package (e.g., 'Maldives')."
                },
                "dates": {
                    "type": "string",
                    "description": "The dates for the vacation package in YYYY-MM-DD format."
                },
                "customer_name": {
                    "type": "string",
                    "description": "The name of the customer making the booking."
                },
                "email": {
                    "type": "string",
                    "format": "email",
                    "description": "The email address of the customer."
                },
                "phone": {
                    "type": "string",
                    "description": "The phone number of the customer."
                }
            },
            "required": ["destination", "dates", "customer_name", "email", "phone"]
        }
    }
]


## Step 3: Simulate Synthetic Tool Responses

### Description
Simulate responses for the tools to mimic real-world interactions with booking systems and databases.

### Functions
- **check_availability:** Simulates availability checks for different destinations and dates.
- **book_vacation:** Simulates the booking process for available vacation packages.

In [13]:
# Step 3: Simulate synthetic tool responses
def check_availability(destination, dates):
    # Simulated availability check
    if destination.lower() == "maldives" and dates == "2024-07-01":
        return True
    elif destination.lower() == "bahamas" and dates == "2024-08-15":
        return True
    else:
        return False

def book_vacation(destination, dates, customer_name, email, phone):
    # Simulated booking process
    if destination.lower() == "maldives" and dates == "2024-07-01":
        booking_id = "BWV123"  # Example booking ID
        return {"booking_id": booking_id}
    elif destination.lower() == "bahamas" and dates == "2024-08-15":
        booking_id = "BWV456"  # Example booking ID
        return {"booking_id": booking_id}
    else:
        return "Booking failed due to unavailable dates or destination."

## Step 4: Process Tool Calls and Return Results

### Description
Create a function to process tool calls made by the chatbot and return the appropriate results.

### Function
- **process_tool_call:** Processes tool calls and returns results based on the input parameters.


In [ ]:
# Step 4: Process tool calls and return results
def process_tool_call(tool_name, tool_input):
    if tool_name == "check_availability":
        return check_availability(tool_input["destination"], tool_input["dates"])
    elif tool_name == "book_vacation":
        return book_vacation(tool_input["destination"], tool_input["dates"], tool_input["customer_name"], tool_input["email"], tool_input["phone"])

## Step 5: Interact with the Chatbot

### Description
Implement the interaction with the chatbot, where users can request to book holiday vacations.

### Function
- **book_holiday_vacation:** Initiates the conversation with the chatbot and handles tool calls and responses.


In [16]:
# Step 5: Interact with the chatbot
import json

def book_holiday_vacation(destination, dates, customer_name, email, phone):
    print(f"\n{'='*50}\nBooking Holiday Vacation to {destination}\n{'='*50}")

    messages = [
        {"role": "user", "content": f"I'd like to book a holiday vacation to {destination} from {dates} with Bluewave Holidays."}
    ]

    response = client.beta.tools.messages.create(
        model=MODEL_NAME,
        max_tokens=4096,
        tools=tools,
        messages=messages
    )

    print(f"\nInitial Response:")
    print(f"Stop Reason: {response.stop_reason}")
    print(f"Content: {response.content}")

    while response.stop_reason == "tool_use":
        tool_use = next(block for block in response.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input

        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input:")
        print(json.dumps(tool_input, indent=2))

        tool_result = process_tool_call(tool_name, tool_input)

        print(f"\nTool Result:")
        print(json.dumps(tool_result, indent=2))

        messages = [
            {"role": "user", "content": f"I'd like to book a holiday vacation to {destination} from {dates} with Bluewave Holidays."},
            {
                "role": "user",
                "content": [
                    {
                        "type": "tool_result",
                        "tool_use_id": tool_use.id,
                        "content": str(tool_result),
                    }
                ],
            },
        ]

        response = client.beta.tools.messages.create(
            model=MODEL_NAME,
            max_tokens=4096,
            tools=tools,
            messages=messages
        )

        print(f"\nResponse:")
        print(f"Stop Reason: {response.stop_reason}")
        print(f"Content: {response.content}")

    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )

    print(f"\nFinal Response: {final_response}")

    return final_response

## Step 6: Test Booking the Holiday Vacation

### Description
Test the functionality of the chatbot by booking holiday vacations to different destinations.

### Example Calls
- `book_holiday_vacation("Maldives", "2024-07-01", "John Doe", "john@example.com", "123-456-7890")`
- `book_holiday_vacation("Bahamas", "2024-08-15", "Jane Smith", "jane@example.com", "987-654-3210")`


In [17]:
# Step 6: Test booking the holiday vacation
book_holiday_vacation("Maldives", "2024-07-01", "John Doe", "john@example.com", "123-456-7890")
book_holiday_vacation("Bahamas", "2024-08-15", "Jane Smith", "jane@example.com", "987-654-3210")


Booking Holiday Vacation to Maldives

Initial Response:
Stop Reason: end_turn
Content: [TextBlock(text='<thinking>\nBased on the user\'s request, the relevant tool is book_vacation. This function requires the following parameters:\n\nRequired parameters:\n- destination: The user specified "Maldives"\n- dates: The user provided "2024-07-01", but this is only a single date. We would need a date range to book the vacation.\n- customer_name: Not provided \n- email: Not provided\n- phone: Not provided\n\nThe destination and a start date were provided, but we are missing the end date, customer name, email and phone number which are all required to book the vacation.\n</thinking>\n\nTo book your vacation to the Maldives, I will need some additional information:\n\n- The end date for your trip (you provided a start date of 2024-07-01 but I need to know what day you plan to return)\n- Your full name \n- Your email address\n- Your phone number\n\nPlease provide those details and I\'ll be happy 

'<thinking>\nThe user has requested to book a holiday vacation package to Bahamas starting from 2024-08-15. \n\nThe relevant tool to use for this request is book_vacation, which requires the following parameters:\n- destination (provided by user - "Bahamas") \n- dates (provided by user - "2024-08-15", but end date is missing)\n- customer_name (not provided by user)\n- email (not provided by user)\n- phone (not provided by user)\n\nSince the end date and several other required parameters for booking are missing, I cannot proceed with calling the book_vacation function yet. I need to ask the user to provide the missing information before I can book their vacation package.\n</thinking>\n\nTo book your Bahamas vacation, I will need some additional information:\n\n- What is the end date of your trip? You provided the start date of 2024-08-15 but I also need to know when you plan to return.\n- What is your full name for the booking? \n- What is your email address?\n- What is your phone numbe